## Setting LLM Model

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
#print("Open AI - ",os.getenv("LITELLM_URL"),os.getenv("OPENAI_API_MODEL"), os.getenv("OPENAI_API_EMBEDDING"))
#print("OLLAMA  - ",os.getenv("OLLAMA_URL"),os.getenv("OLLAMA_MODEL"))
#print("Local OLLAMA - ",os.getenv("OLLAMA_LOCAL_URL"),os.getenv("OLLAMA_LOCAL_MODEL"))

True

In [6]:
from llama_index.core import Settings

### RUN LLM AS OLLAMA 

In [ ]:
#install OPEN AI LLM, skip if already installed
!pipenv install llama-index-llms-ollama

In [7]:
from llama_index.llms.ollama import Ollama
api_base=os.getenv("OLLAMA_LOCAL_URL")
model=os.getenv("OLLAMA_LOCAL_MODEL")
Settings.llm = Ollama(model=model, base_url=api_base,request_timeout=120.0)

# use remote ollam
"""
api_base=os.getenv("OLLAMA_URL")
model=os.getenv("OLLAMA_MODEL")
llm = Ollama(model=model, base_url=api_base,request_timeout=180.0)
"""
#test run
#response = llm.complete("What is the capital of France?")
#print(response)

'\napi_base=os.getenv("OLLAMA_URL")\nmodel=os.getenv("OLLAMA_MODEL")\nllm = Ollama(model=model, base_url=api_base,request_timeout=180.0)\n'

### RUN LLM AS OPEN AI 

In [ ]:
#install OPEN AI LLM, skip if already installed
!pipenv install llama-index-llms-openai

In [5]:
from llama_index.llms.openai import OpenAI
api_base=os.getenv("LITELLM_URL")
model=os.getenv("OPENAI_API_MODEL")

Settings.llm = OpenAI(
    model=model,
    api_base = api_base,
    temperature=0.3
)

resp = Settings.llm.complete("What is the capital of France?")
print(resp)

The capital of France is Paris.


## Setting Embedding Model

In [6]:
# use open AI embedding
from llama_index.embeddings.openai import OpenAIEmbedding
api_base=os.getenv("LITELLM_URL")
embedding_model=os.getenv("OPENAI_API_EMBEDDING")

Settings.embed_model = OpenAIEmbedding(
    model_name=embedding_model,
    api_base = api_base,
)

# embed_text = Settings.embed_model.get_text_embedding("hello")
# print(f"{len(embed_text)}, {embed_text}")

In [8]:
# use other embedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
#embed_text = Settings.embed_model.get_text_embedding("hello")
#print(f"{len(embed_text)}, {embed_text}")

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/paul").load_data()

In [10]:
import logging
import sys

#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

### KeywordTableIndex

In [11]:
from llama_index.core import KeywordTableIndex

index = KeywordTableIndex.from_documents(documents)

In [12]:
query = index.as_query_engine(llm=Settings.llm)
print(query.query("language"))

Lisp 



### VectorStoreIndex

In [13]:
index = VectorStoreIndex.from_documents(documents)


In [14]:
query = index.as_query_engine(llm=Settings.llm)
print(query.query("Who is Paul Graha?"))

He wrote a novel about an intelligent computer called Mike.  





In [20]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own.\n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)
refine_template = PromptTemplate(refine_template_str)

In [21]:
print(text_qa_template)

metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>} template_vars=['context_str', 'query_str'] kwargs={} output_parser=None template_var_mappings=None function_mappings=None template="Context information is below.\n---------------------\n{context_str}\n---------------------\nUsing both the context information and also using your own knowledge, answer the question: {query_str}\nIf the context isn't helpful, you can also answer the question on your own.\n"


In [ ]:
print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=Settings.llm,
    ).query("Who is Joe Biden?")
)

### Tree Index

In [23]:
from llama_index.core import TreeIndex

In [ ]:
index = TreeIndex.from_documents(documents)
query = index.as_query_engine(llm=Settings.llm)
print(query.query("Who is Paul Graha?"))

In [26]:
from llama_index.core import Document

In [27]:
print(Document.example())

Doc ID: e8481d85-7287-4aba-89cc-079e8643ba2c
Text: Context LLMs are a phenomenal piece of technology for knowledge
generation and reasoning. They are pre-trained on large amounts of
publicly available data. How do we best augment LLMs with our own
private data? We need a comprehensive toolkit to help perform this
data augmentation for LLMs.  Proposed Solution That's where LlamaIndex
comes in. Ll...
